In [ ]:
import pandas as pd
import numpy as np
import os
print('number of training image')
print(len([name for name in os.listdir('/kaggle/input/plant-pathology-2021-fgvc8/train_images/')]))
print(len([name for name in os.listdir('/kaggle/input/plant-pathology-2021-fgvc8/test_images/')]))
training_csv = pd.read_csv('../input/plant-pathology-2021-fgvc8/train.csv')
training_class = np.array([])
for labels in pd.unique(training_csv['labels']):
    training_class = np.append(training_class,labels.split())
training_class = np.unique(training_class)
print('\nnumber of class')
print(training_class)
print(len(training_class))

In [ ]:
def predict2strings(pred, threshold):
    t = np.array(threshold)
    return ' '.join(training_class[np.where(pred >= t)])

def predict2n_hot(pred, threshold):
    t = np.array(threshold)
    return np.where(pred >= t,np.ones(pred.shape),np.zeros(pred.shape))

def n_hot2string(n_hot):
    return ' '.join(training_class[np.where(n_hot >= 1)])

In [ ]:
import tensorflow as tf
model_input = tf.keras.layers.Input(shape=(299, 299, 3))
xception_layer = tf.keras.applications.Xception(
    include_top=False,
    weights=None,
    input_shape= (299, 299, 3),
    input_tensor= model_input,
    pooling = 'max'
).output
fc_1 = tf.keras.layers.Dense(1024)(xception_layer)
fc_2 = tf.keras.layers.Dense(512)(fc_1)
model_output = tf.keras.layers.Dense(len(training_class),activation = 'sigmoid')(fc_2)
model = tf.keras.Model(inputs = model_input,outputs = model_output)

In [ ]:
model.load_weights('../input/cs5489-project-train-baseline/my_model/model_1')

In [ ]:
threshold = [0.5469251871109009, 0.4226226806640625, 0.9167647957801819, 0.1412835568189621, 0.22426636517047882, 0.4077064096927643]
test_img_path = '../input/plant-pathology-2021-fgvc8/test_images/'
def write_csv_kaggle_tags():
    import csv
    tmp = [['image', 'labels']]
    
    for path in os.listdir(test_img_path):
        image = tf.keras.preprocessing.image.load_img(test_img_path + path, target_size=(299,299))
        image = tf.keras.preprocessing.image.img_to_array(image)
        image = tf.image.per_image_standardization(image)
        image = tf.expand_dims(image, axis=0,)
        test_predscore = model(image)[0]
        row = [path, predict2strings(test_predscore, threshold = threshold)]
        tmp.append(row)
    f = open("submission.csv", 'w')
    tmp
    writer = csv.writer(f)
    writer.writerows(tmp)
    f.close()

In [ ]:
write_csv_kaggle_tags()